In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

train_data = pd.read_csv('sign_mnist_train.csv')
test_data = pd.read_csv('sign_mnist_test.csv')
print(train_data.isna().sum())
print(test_data.isna().sum())

train_labels = train_data['label'].values
train_features = train_data.drop('label', axis=1).values
test_labels = test_data['label'].values
test_features = test_data.drop('label', axis=1).values
train_features = train_features / 255.0
test_features = test_features / 255.0
train_features = train_features.reshape(-1, 28, 28, 1)
test_features = test_features.reshape(-1, 28, 28, 1)
lb = LabelBinarizer()
train_labels = lb.fit_transform(train_labels)
test_labels = lb.transform(test_labels)
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(len(lb.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(datagen.flow(X_train, y_train, batch_size=32),
          validation_data=(X_val, y_val),
          epochs=20)

model.save('image_classification_model.h5')

test_loss, test_accuracy = model.evaluate(test_features, test_labels)
print(f'Test accuracy: {test_accuracy}')


label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64
label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64
Epoch 1/20
687/687 [==============================] - 31s 44ms/step - loss: 3.0199 - accuracy: 0.1086 - val_loss: 2.2290 - val_accuracy: 0.3732
Epoch 2/20
687/687 [==============================] - 31s 45ms/step - loss: 2.3621 - accuracy: 0.2886 - val_loss: 1.5854 - val_accuracy: 0.6046
Epoch 3/20
687/687 [==============================] - 31s 46ms/step - loss: 2.0546 - accuracy: 0.3855 - val_loss: 1.2654 - val_accuracy: 0.7174
Epoch 4/20
687/687 [==============================] - 31s 45ms/step - loss: 1.8986 - accuracy: 0.4378 - val_loss: 1.1319 - val_accuracy: 0.7485
Epoch 5/20
687/687 [==============================] - 31s 45ms/s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


225/225 [==============================] - 2s 8ms/step - loss: 0.6884 - accuracy: 0.8886
Test accuracy: 0.8885945081710815
